# Comparison of MLE models for *N. gonorrhoeae* WHO M treated with Ciprofloxacin

In [11]:
import os

import chi
import chi.plots
import numpy as np
import pandas as pd
import plotly.colors
import plotly.graph_objects as go
from scipy.stats import lognorm
import xarray as xr

## Show data

In [3]:
# Get data
directory = os.path.dirname(os.path.dirname(os.getcwd()))
data = pd.read_csv(directory + '/data/who_m_ciprofloxacin.csv')

# Create scatter plots of data
fig = chi.plots.PDTimeSeriesPlot()
fig.add_data(data, biomarker='Bacterial count')
fig.set_axis_labels(xlabel='Time in hours', ylabel='Bacterial count in CFU/mL')
fig.show()

Export figure for publication

In [7]:
# Shift the time, such that first dose is at 0
temp = data.copy()
temp.Time = temp.Time - 4

fig = chi.plots.PDTimeSeriesPlot()
fig.add_data(temp, biomarker='Bacterial count')
fig.set_axis_labels(xlabel='Time in hours', ylabel='Bacterial count in CFU/mL')
fig._fig.layout.updatemenus = []
concs = [0.003, 0.031, 0.063, 0.125, 0.25, 0.5, 1, 2, 4, 8, 16, 32]
colours = plotly.colors.sequential.ice
for _id, scatter in enumerate(fig._fig.data):
    scatter.mode = 'markers+lines'
    scatter.line.color = colours[_id]
    scatter.marker.color = colours[_id]
    scatter.name = str(concs[_id])
    # scatter.showlegend = False

# Add a line indicating the dosing regimen
fig._fig.add_shape(
    type="line",
    x0=0, x1=6,
    yref="paper",
    y0=0.99, y1=0.99,
    line=dict(color='Black')
)
fig._fig.add_shape(
    type="line",
    x0=0, x1=0,
    yref="paper",
    y0=0.98, y1=1,
    line=dict(color='Black')
)
fig._fig.add_shape(
    type="line",
    x0=6, x1=6,
    yref="paper",
    y0=0.98, y1=1,
    line=dict(color='Black')
)

fig._fig.update_layout(
    width=500,
    height=500,
    yaxis=dict(
        type='log',
    ),
    font=dict(size=20),
    legend=dict(
        title='Conc.\n[mg/L]',
        font=dict(size=16)
    )
)

directory = os.getcwd()
fig._fig.write_image(directory + '/in_vitro_data.pdf', width=500, height=500)
fig.show()

## Estimate model likelihoods

Uses AIC scores estimated in

- `infer_K_model.ipynb`
- `infer_KP_model.ipynb`
- `infer_KR_model.ipynb`.

The model likelihoods are defined as

$$
    w_m =
        \frac{\mathrm{e}^{-\mathrm{AIC}_m / 2}}
        {\sum _m \mathrm{e}^{-\mathrm{AIC}_m / 2}},
$$

where $m$ labels the candidate models.

In [10]:
# Load elpd estimates from disk
directory = os.getcwd()
models = ['K_model', 'KP_model', 'KR_model']
n_models = len(models)
aic_scores = np.empty(shape=(n_models))
for id_m, model in enumerate(models):
    # Get AIC scores
    path = os.path.join(
        directory, 'derived_data/%s_inference_data.nc' % model)
    inf_data = xr.load_dataarray(path)
    aic_scores[id_m] = inf_data.attrs['AIC score']

# Compute elpd deltas to avoid numerical overflow
delta_aic = aic_scores - aic_scores[-1]

# Compute weights
weights_aic = np.exp(-delta_aic / 2) / np.sum(np.exp(-delta_aic / 2), axis=0)

# Create figure
fig = go.Figure()
colors = plotly.colors.qualitative.Plotly
fig.add_trace(go.Bar(
    x=['K', 'KP', 'KR'],
    y=weights_aic,
    marker_color=colors[:n_models]
))
fig.update_traces(
    marker_line_color='black',
    marker_line_width=1.5)
fig.update_layout(
    template='plotly_white',
    yaxis_title='Weight',
    showlegend=False,
    yaxis_range=[0, 1],
    font=dict(size=20),
    xaxis_showticklabels=True
)

directory = os.getcwd()
fig.write_image(
    directory + '/model_weights_in_vitro_study.pdf', width=500, height=500)
fig.show()

## Predict treatment response with likely models for clinical dosing regimen

In [21]:
# Define KP model
directory = os.path.dirname(os.path.dirname(os.getcwd()))
kp_model = chi.PharmacokineticModel(directory + '/models/KP_model.xml')
kp_model.set_administration(compartment='central', direct=False)
kp_model.set_parameter_names(names={
    'myokit.bacterial_count_susceptible': 'Initial bacterial count in CFU/ml',
    'myokit.death_rate': 'Death rate in 1/h',
    'myokit.growth_rate': 'Growth rate in 1/h',
    'myokit.kappa': 'Kill rate in ml/ng/h',
    'myokit.transition_rate_12': 'Transition rate to dividing in 1/h',
    'myokit.transition_rate_21': 'Transition rate to non-dividing in ml/ng/h'})
kp_model.set_outputs(['myokit.total_bacterial_count'])
kp_model.set_output_names({
    'myokit.total_bacterial_count': 'Bacterial count in CFU/ml'})
kp_model = chi.ReducedMechanisticModel(
    kp_model)
kp_model.fix_parameters({
    'central.drug_amount': 0,
    'dose.drug_amount': 0,
    'myokit.bacterial_count_adapted': 0,
    'dose.absorption_rate': 2.7,           # in 1/h, Sanchez et al
    'central.size': 3.7 * 70,              # in L/kg, Sanchez et al for 70 kg
    'myokit.elimination_rate': 0.17,       # in 1/h, Sanchez et al
    'myokit.gamma': 1})

# Define KR model
kr_model = chi.PharmacokineticModel(directory + '/models/KR_model.xml')
kr_model.set_administration(compartment='central', direct=False)
kr_model.set_parameter_names(names={
    'myokit.bacterial_count_susceptible': 'Initial bacterial count in CFU/ml',
    'myokit.concentration_e50_adapted': 'Adapted EC 50 in ng/ml',
    'myokit.growth_rate': 'Growth rate in 1/h',
    'myokit.kappa_adapted': 'Adapted max. kill rate in 1/h',
    'myokit.kappa_susceptible': 'Wild type kill rate in ml/ng/h',
    'myokit.mutation_rate': 'Mutation rate in ml/ng/h'})
kr_model.set_outputs(['myokit.total_bacterial_count'])
kr_model.set_output_names({
    'myokit.total_bacterial_count': 'Bacterial count in CFU/ml'})
kr_model = chi.ReducedMechanisticModel(
    kr_model)
kr_model.fix_parameters({
    'central.drug_amount': 0,
    'dose.drug_amount': 0,
    'myokit.bacterial_count_adapted': 0,
    'dose.absorption_rate': 2.7,           # in 1/h, Sanchez et al
    'central.size': 3.7 * 70,              # in L/kg, Sanchez et al for 70 kg
    'myokit.elimination_rate': 0.17,       # in 1/h, Sanchez et al
    'myokit.gamma': 1})

# Load MLE parameters
directory = os.getcwd()
models = ['KP_model', 'KR_model']
parameters = []
for model in models:
    inf_data = xr.load_dataarray(
        directory + '/derived_data/%s_inference_data.nc' % model)
    parameters.append(inf_data.values)

# Set dosing regimen (250 mg twice a day, Sanchez et. al.)
kp_model.set_dosing_regimen(dose=250E3, start=4, period=12)
kr_model.set_dosing_regimen(dose=250E3, start=4, period=12)

# Simulate treatment response
times = np.linspace(0, 40, 500)
sim_kp = kp_model.simulate(parameters=parameters[0][:-1], times=times)[0]
sim_kr = kr_model.simulate(parameters=parameters[1][:-1], times=times)[0]

# Visualise prediction
fig = go.Figure()
colors = plotly.colors.qualitative.Plotly

# Shift times to make start of first dose to coincide with 0
times -= 4

# Plot KP prediction
sigma = parameters[0][-1]
lower = sim_kp * lognorm.ppf(0.05, s=sigma, scale=np.exp(-sigma**2/2))
upper = sim_kp * lognorm.ppf(0.95, s=sigma, scale=np.exp(-sigma**2/2))
fig.add_trace(go.Scatter(
    x=np.concatenate((times, times[::-1])),
    y=np.concatenate((lower, upper[::-1])),
    name='KP',
    legendgroup='KP',
    fill='toself',
    opacity=0.5,
    showlegend=False,
    line=dict(color='grey', width=1)
))
fig.add_trace(go.Scatter(
    x=times,
    y = sim_kp,
    name='KP',
    legendgroup='KP',
    line=dict(color=colors[1])
))

# Plot KR prediction
sigma = parameters[1][-1]
lower = sim_kr * lognorm.ppf(0.05, s=sigma, scale=np.exp(-sigma**2/2))
upper = sim_kr * lognorm.ppf(0.95, s=sigma, scale=np.exp(-sigma**2/2))
fig.add_trace(go.Scatter(
    x=np.concatenate((times, times[::-1])),
    y=np.concatenate((lower, upper[::-1])),
    name='KR',
    legendgroup='KR',
    fill='toself',
    opacity=0.5,
    showlegend=False,
    line=dict(color='grey', width=1)
))
fig.add_trace(go.Scatter(
    x=times,
    y = sim_kr,
    name='KR',
    legendgroup='KR',
    line=dict(color=colors[2])
))

# Add lines to indicate time point of dose administration
fig.add_shape(
    type="line",
    x0=0, x1=0,
    yref="paper",
    y0=0.98, y1=1,
    line=dict(color='Black')
)
fig.add_shape(
    type="line",
    x0=12, x1=12,
    yref="paper",
    y0=0.98, y1=1,
    line=dict(color='Black')
)
fig.add_shape(
    type="line",
    x0=24, x1=24,
    yref="paper",
    y0=0.98, y1=1,
    line=dict(color='Black')
)

# Set layout
fig.update_layout(
    template='plotly_white',
    xaxis_title='Time in hours',
    yaxis_title='Bacterial count in CFU/mL',
    yaxis_type='log',
    yaxis_range=[0, 8],
    legend=dict(
        yanchor="top",
        y=0.9,
        xanchor="left",
        x=0.8,
        bordercolor="Black",
        borderwidth=1
    ),
    font=dict(size=20),
)

directory = os.getcwd()
fig.write_image(directory + '/MLE_predictions.pdf', width=500, height=500)
fig.show()